In [9]:
from __future__ import annotations
import typing
import sympy

class BaseNode:
    def __init__(self, *args: int | BaseNode | typing.Type[BaseNode]):
        assert all(
            (
                isinstance(arg, BaseNode)
                or isinstance(arg, int)
                or (isinstance(arg, type) and issubclass(arg, BaseNode))
            )
            for arg in args)
        self._args = args

    @property
    def args(self) -> tuple[int | BaseNode | typing.Type[BaseNode], ...]:
        return self._args

    @property
    def func(self) -> typing.Type[typing.Self]:
        return type(self)

    def __eq__(self, other) -> bool:
        if not isinstance(other, BaseNode):
            return False
        if not self.func == other.func:
            return False
        if not len(self.args) == len(other.args):
            return False
        for i, arg in enumerate(self.args):
            if arg != other.args[i]:
                return False
        return True

    def eval(self) -> sympy.Basic:
        name = self.func.__name__
        if len(self.args) == 0:
            return sympy.Symbol(name)

        def eval_arg(arg: int | BaseNode | typing.Type[BaseNode]):
            if isinstance(arg, BaseNode):
                return arg.eval()
            if isinstance(arg, int):
                return sympy.Integer(arg)
            if isinstance(arg, type) and issubclass(arg, BaseNode):
                return sympy.Symbol(arg.__name__)
            raise ValueError(f"Invalid argument: {arg}")

        fn = sympy.Function(name)
        args: list[sympy.Basic] = [eval_arg(arg) for arg in self.args]
        return fn(*args)

BaseNode(1, BaseNode(456), BaseNode).eval()

BaseNode(1, BaseNode(456), BaseNode)

In [10]:
BaseNode(1, BaseNode(456), BaseNode) == BaseNode(1, BaseNode(456), BaseNode)

True

In [12]:
BaseNode(1, BaseNode(4567), BaseNode) == BaseNode(1, BaseNode(456), BaseNode)

False

In [13]:
sympy.Function('f')(1) == sympy.Function('f')(1)

True

In [14]:
sympy.Function('f')(1) == sympy.Function('g')(1)

False